In [1]:
import glob
import os
import pandas as pd

PATH = "data/OpenClinica_core_src_main_java/"
FILE_MATCHER = "**/*."
FILE_EXTENSION = "java"
PATH_MATCHER = os.path.join(PATH, FILE_MATCHER) + FILE_EXTENSION
FILES_TO_EXCLUDE = ["package-info.java"]
corpus = pd.DataFrame(glob.glob(PATH_MATCHER, recursive=True), columns=["filepath"])
corpus['filename'] = corpus['filepath'].str.split(os.path.sep).str[-1]
corpus['typename'] = corpus['filename'].str.replace("." + FILE_EXTENSION, "", regex=False)
corpus = corpus[~corpus["filename"].isin(FILES_TO_EXCLUDE)].copy()
corpus.head()

,filepath,filename,typename
0,data/OpenClinica_core_src_main_java/org/akaza/...,AuditBean.java,AuditBean
1,data/OpenClinica_core_src_main_java/org/akaza/...,AuditEventBean.java,AuditEventBean
2,data/OpenClinica_core_src_main_java/org/akaza/...,CRFBean.java,CRFBean
3,data/OpenClinica_core_src_main_java/org/akaza/...,DeletedEventCRFBean.java,DeletedEventCRFBean
4,data/OpenClinica_core_src_main_java/org/akaza/...,DisplayStudyBean.java,DisplayStudyBean


In [2]:
def read_content(path):
    with open(path) as source_code:
        return source_code.read()

corpus['code'] = corpus['filepath'].apply(read_content)
corpus.head()

,filepath,filename,typename,code
0,data/OpenClinica_core_src_main_java/org/akaza/...,AuditBean.java,AuditBean,package org.akaza.openclinica.bean.admin;\n\ni...
1,data/OpenClinica_core_src_main_java/org/akaza/...,AuditEventBean.java,AuditEventBean,/*\n * OpenClinica is distributed under the\n ...
2,data/OpenClinica_core_src_main_java/org/akaza/...,CRFBean.java,CRFBean,/*\n * OpenClinica is distributed under the\n ...
3,data/OpenClinica_core_src_main_java/org/akaza/...,DeletedEventCRFBean.java,DeletedEventCRFBean,package org.akaza.openclinica.bean.admin;\n\ni...
4,data/OpenClinica_core_src_main_java/org/akaza/...,DisplayStudyBean.java,DisplayStudyBean,/*\n * OpenClinica is distributed under the\n ...


In [3]:
corpus['lines'] = corpus['code'].str.count("\n")
corpus.head()

,filepath,filename,typename,code,lines
0,data/OpenClinica_core_src_main_java/org/akaza/...,AuditBean.java,AuditBean,package org.akaza.openclinica.bean.admin;\n\ni...,241
1,data/OpenClinica_core_src_main_java/org/akaza/...,AuditEventBean.java,AuditEventBean,/*\n * OpenClinica is distributed under the\n ...,320
2,data/OpenClinica_core_src_main_java/org/akaza/...,CRFBean.java,CRFBean,/*\n * OpenClinica is distributed under the\n ...,126
3,data/OpenClinica_core_src_main_java/org/akaza/...,DeletedEventCRFBean.java,DeletedEventCRFBean,package org.akaza.openclinica.bean.admin;\n\ni...,63
4,data/OpenClinica_core_src_main_java/org/akaza/...,DisplayStudyBean.java,DisplayStudyBean,/*\n * OpenClinica is distributed under the\n ...,55


In [4]:
import re
import os.path
from pygments.token import Token
from pygments.lexers.jvm import JavaLexer

DOT_PATTERN = re.compile(r'(.*)\.(.*)')
CAMEL_CASE_1_PATTERN = re.compile(r'(.)([A-Z][a-z]+)')
CAMEL_CASE_2_PATTERN = re.compile(r'([a-z0-9])([A-Z])')
WORD_BOUNDARY_PATTERN = re.compile(r'[^a-zA-Z]')

JAVA_STOP_WORDS = set(["byte", "short", "int", "long", "float", "double", "char", "string", "object", "java"])
PRODUCT_NAME_STOP_WORDS = set(["openclinica"])
START_NAMES_TO_AVOID = set(["package", "import"])
STOP_LIST = JAVA_STOP_WORDS | PRODUCT_NAME_STOP_WORDS


def extract_identifiers(source_code, lexer, min_token_length):

    identifiers = []
    
    filtered_source_code = ""
    
    for line in source_code.split("\n"):
        beginning = line.split(" ", maxsplit=1)[0]
        if beginning not in START_NAMES_TO_AVOID:
            filtered_source_code += line
               
    for token_type, value in lexer.get_tokens(filtered_source_code):

        if token_type in Token.Name or Token:
            tokens = str(value)
            tokens = CAMEL_CASE_1_PATTERN.sub(r'\1 \2', tokens)
            tokens = CAMEL_CASE_2_PATTERN.sub(r'\1 \2', tokens)
            tokens = WORD_BOUNDARY_PATTERN.sub(' ', tokens)
            tokens = tokens.split(' ')

            for token in tokens:
                if len(token) >= min_token_length and token.lower() not in STOP_LIST:
                    identifiers.append(token.lower())

    return " ".join(identifiers)

LEXER = JavaLexer()
MIN_TOKEN_LENGTH = 3
# 
def extract_java_identifiers(source_code):
    return extract_identifiers(source_code, LEXER, MIN_TOKEN_LENGTH)

corpus['tokens'] = corpus['code'].apply(extract_java_identifiers)
corpus.head()

,filepath,filename,typename,code,lines,tokens
0,data/OpenClinica_core_src_main_java/org/akaza/...,AuditBean.java,AuditBean,package org.akaza.openclinica.bean.admin;\n\ni...,241,audit bean entity bean date audit date audit t...
1,data/OpenClinica_core_src_main_java/org/akaza/...,AuditEventBean.java,AuditEventBean,/*\n * OpenClinica is distributed under the\n ...,320,audit event bean auditable entity bean
2,data/OpenClinica_core_src_main_java/org/akaza/...,CRFBean.java,CRFBean,/*\n * OpenClinica is distributed under the\n ...,126,crf bean auditable entity bean status descript...
3,data/OpenClinica_core_src_main_java/org/akaza/...,DeletedEventCRFBean.java,DeletedEventCRFBean,package org.akaza.openclinica.bean.admin;\n\ni...,63,deleted event crf bean study event crf name cr...
4,data/OpenClinica_core_src_main_java/org/akaza/...,DisplayStudyBean.java,DisplayStudyBean,/*\n * OpenClinica is distributed under the\n ...,55,display study bean auditable entity bean study...


In [5]:
corpus['tokens_count'] = corpus['tokens'].apply(len)
corpus.head(15)

,filepath,filename,typename,code,lines,tokens,tokens_count
0,data/OpenClinica_core_src_main_java/org/akaza/...,AuditBean.java,AuditBean,package org.akaza.openclinica.bean.admin;\n\ni...,241,audit bean entity bean date audit date audit t...,2477
1,data/OpenClinica_core_src_main_java/org/akaza/...,AuditEventBean.java,AuditEventBean,/*\n * OpenClinica is distributed under the\n ...,320,audit event bean auditable entity bean,38
2,data/OpenClinica_core_src_main_java/org/akaza/...,CRFBean.java,CRFBean,/*\n * OpenClinica is distributed under the\n ...,126,crf bean auditable entity bean status descript...,69
3,data/OpenClinica_core_src_main_java/org/akaza/...,DeletedEventCRFBean.java,DeletedEventCRFBean,package org.akaza.openclinica.bean.admin;\n\ni...,63,deleted event crf bean study event crf name cr...,589
4,data/OpenClinica_core_src_main_java/org/akaza/...,DisplayStudyBean.java,DisplayStudyBean,/*\n * OpenClinica is distributed under the\n ...,55,display study bean auditable entity bean study...,234
5,data/OpenClinica_core_src_main_java/org/akaza/...,JDBCType.java,JDBCType,package org.akaza.openclinica.bean.admin;\n\n\...,184,jdbc type bit types bit tinyint types tinyint ...,1060
6,data/OpenClinica_core_src_main_java/org/akaza/...,NewCRFBean.java,NewCRFBean,/*\n * OpenClinica is distributed under the\n ...,821,new crf bean serializable data source dao dige...,293
7,data/OpenClinica_core_src_main_java/org/akaza/...,QueryObject.java,QueryObject,package org.akaza.openclinica.bean.admin;\n\ni...,30,query sql array list sql parameter sql paramet...,255
8,data/OpenClinica_core_src_main_java/org/akaza/...,SqlParameter.java,SqlParameter,package org.akaza.openclinica.bean.admin;\n\n\...,41,sql parameter value jdbc type type sql paramet...,268
9,data/OpenClinica_core_src_main_java/org/akaza/...,StudyEventAuditBean.java,StudyEventAuditBean,/*\n * OpenClinica is distributed under the\n ...,101,study event audit bean study event definition ...,944


In [6]:
# some files seem to have no tokens, problem needs to be analyzed further!
corpus = corpus[corpus['tokens_count'] != 0].copy()

In [7]:
corpus.to_csv("temp/openclinica_corpus.bz2", columns=["filepath", "typename", "tokens"], compression='bz2')

In [8]:
import pandas as pd
corpus = pd.read_csv('temp/openclinica_corpus.bz2', index_col=0)
corpus.head()

,filepath,typename,tokens
0,data/OpenClinica_core_src_main_java/org/akaza/...,AuditBean,audit bean entity bean date audit date audit t...
1,data/OpenClinica_core_src_main_java/org/akaza/...,AuditEventBean,audit event bean auditable entity bean
2,data/OpenClinica_core_src_main_java/org/akaza/...,CRFBean,crf bean auditable entity bean status descript...
3,data/OpenClinica_core_src_main_java/org/akaza/...,DeletedEventCRFBean,deleted event crf bean study event crf name cr...
4,data/OpenClinica_core_src_main_java/org/akaza/...,DisplayStudyBean,display study bean auditable entity bean study...


In [9]:
corpus['splitted_tokens'] = corpus['tokens'].str.split()
corpus[:5]

,filepath,typename,tokens,splitted_tokens
0,data/OpenClinica_core_src_main_java/org/akaza/...,AuditBean,audit bean entity bean date audit date audit t...,"[audit, bean, entity, bean, date, audit, date,..."
1,data/OpenClinica_core_src_main_java/org/akaza/...,AuditEventBean,audit event bean auditable entity bean,"[audit, event, bean, auditable, entity, bean]"
2,data/OpenClinica_core_src_main_java/org/akaza/...,CRFBean,crf bean auditable entity bean status descript...,"[crf, bean, auditable, entity, bean, status, d..."
3,data/OpenClinica_core_src_main_java/org/akaza/...,DeletedEventCRFBean,deleted event crf bean study event crf name cr...,"[deleted, event, crf, bean, study, event, crf,..."
4,data/OpenClinica_core_src_main_java/org/akaza/...,DisplayStudyBean,display study bean auditable entity bean study...,"[display, study, bean, auditable, entity, bean..."


In [10]:
tokens_list = corpus['tokens'].str.split()
tokens_list[5:]

5      [jdbc, type, bit, types, bit, tinyint, types, ...
6      [new, crf, bean, serializable, data, source, d...
7      [query, sql, array, list, sql, parameter, sql,...
8      [sql, parameter, value, jdbc, type, type, sql,...
9      [study, event, audit, bean, study, event, defi...
                             ...                        
732    [item, validator, validator, item, dao, item, ...
733    [response, set, validator, validator, logger, ...
734    [api, security, filter, once, per, request, fi...
735    [open, clinica, jdbc, service, jdbc, dao, impl...
736    [open, clinica, ldap, authorities, populator, ...
Name: tokens, Length: 655, dtype: object

In [11]:
from gensim import corpora
corpus["splitted_tokens"] = corpus['tokens'].str.split()
corpus.head()

,filepath,typename,tokens,splitted_tokens
0,data/OpenClinica_core_src_main_java/org/akaza/...,AuditBean,audit bean entity bean date audit date audit t...,"[audit, bean, entity, bean, date, audit, date,..."
1,data/OpenClinica_core_src_main_java/org/akaza/...,AuditEventBean,audit event bean auditable entity bean,"[audit, event, bean, auditable, entity, bean]"
2,data/OpenClinica_core_src_main_java/org/akaza/...,CRFBean,crf bean auditable entity bean status descript...,"[crf, bean, auditable, entity, bean, status, d..."
3,data/OpenClinica_core_src_main_java/org/akaza/...,DeletedEventCRFBean,deleted event crf bean study event crf name cr...,"[deleted, event, crf, bean, study, event, crf,..."
4,data/OpenClinica_core_src_main_java/org/akaza/...,DisplayStudyBean,display study bean auditable entity bean study...,"[display, study, bean, auditable, entity, bean..."


In [12]:
dictionary = corpora.Dictionary(list(corpus['splitted_tokens'].values))
print(dictionary)

Dictionary<1234 unique tokens: ['audit', 'bean', 'change', 'class', 'code']...>


In [13]:
corpus['token_ids'] = corpus["splitted_tokens"].map(dictionary.doc2bow)
corpus.head()

,filepath,typename,tokens,splitted_tokens,token_ids
0,data/OpenClinica_core_src_main_java/org/akaza/...,AuditBean,audit bean entity bean date audit date audit t...,"[audit, bean, entity, bean, date, audit, date,...","[(0, 37), (1, 4), (2, 7), (3, 2), (4, 2), (5, ..."
1,data/OpenClinica_core_src_main_java/org/akaza/...,AuditEventBean,audit event bean auditable entity bean,"[audit, event, bean, auditable, entity, bean]","[(0, 1), (1, 2), (8, 1), (10, 1), (36, 1)]"
2,data/OpenClinica_core_src_main_java/org/akaza/...,CRFBean,crf bean auditable entity bean status descript...,"[crf, bean, auditable, entity, bean, status, d...","[(1, 2), (5, 1), (8, 1), (36, 1), (37, 1), (38..."
3,data/OpenClinica_core_src_main_java/org/akaza/...,DeletedEventCRFBean,deleted event crf bean study event crf name cr...,"[deleted, event, crf, bean, study, event, crf,...","[(1, 1), (5, 22), (7, 10), (10, 15), (12, 6), ..."
4,data/OpenClinica_core_src_main_java/org/akaza/...,DisplayStudyBean,display study bean auditable entity bean study...,"[display, study, bean, auditable, entity, bean...","[(1, 5), (8, 1), (12, 2), (29, 2), (30, 4), (3..."


In [14]:
id_words = [[(dictionary[id], count) for id, count in line] for line in corpus['token_ids'].values]
print(id_words)

[[('audit', 37), ('bean', 4), ('change', 7), ('class', 2), ('code', 2), ('crf', 28), ('data', 14), ('date', 20), ('entity', 15), ('equals', 2), ('event', 35), ('for', 7), ('get', 23), ('hash', 2), ('interviewed', 7), ('interviewer', 7), ('item', 14), ('key', 7), ('name', 42), ('new', 7), ('obj', 5), ('old', 7), ('ordinal', 7), ('other', 5), ('override', 2), ('prime', 4), ('reason', 7), ('repeat', 7), ('result', 8), ('set', 21), ('study', 7), ('table', 13), ('type', 21), ('user', 17), ('value', 14), ('version', 14)], [('audit', 1), ('bean', 2), ('entity', 1), ('event', 1), ('auditable', 1)], [('bean', 2), ('crf', 1), ('entity', 1), ('auditable', 1), ('array', 1), ('description', 1), ('list', 1), ('status', 1), ('versions', 1)], [('bean', 1), ('crf', 22), ('date', 10), ('event', 15), ('get', 6), ('name', 7), ('set', 6), ('study', 7), ('version', 7), ('deleted', 22)], [('bean', 5), ('entity', 1), ('get', 2), ('set', 2), ('study', 4), ('auditable', 1), ('array', 3), ('list', 3), ('children

In [15]:
id2word = corpora.Dictionary(list(corpus['splitted_tokens'].values))

corpus_new = []
for text in corpus['splitted_tokens'].values:
    new  = id2word.doc2bow(text)
    corpus_new.append(new)

print (corpus_new[0][0:20])

word = id2word[[0][:1][0]]
print (word)



[(0, 37), (1, 4), (2, 7), (3, 2), (4, 2), (5, 28), (6, 14), (7, 20), (8, 15), (9, 2), (10, 35), (11, 7), (12, 23), (13, 2), (14, 7), (15, 7), (16, 14), (17, 7), (18, 42), (19, 7)]
audit


In [16]:
from gensim.models import LdaModel

lda_model = LdaModel(
    corpus = corpus_new,
    num_topics=50,
    id2word=id2word,
    random_state=0,
    passes=10
)
print(lda_model.show_topics(formatted=False))

[(16, [('type', 0.19186799), ('response', 0.14353864), ('data', 0.08264495), ('list', 0.06635962), ('bean', 0.057494223), ('set', 0.055327695), ('item', 0.053487934), ('get', 0.037027813), ('name', 0.030206174), ('present', 0.023433693)]), (38, [('study', 0.17640059), ('subject', 0.0799226), ('dao', 0.07534365), ('bean', 0.054096825), ('get', 0.050389733), ('data', 0.047445957), ('event', 0.047034267), ('source', 0.02643958), ('audit', 0.021604793), ('logs', 0.0186219)]), (15, [('query', 0.13276853), ('get', 0.0889122), ('class', 0.0731385), ('domain', 0.061742578), ('session', 0.05843485), ('hibernate', 0.04075509), ('current', 0.038456574), ('create', 0.03443242), ('find', 0.03176347), ('org', 0.031228721)]), (6, [('set', 0.12511572), ('label', 0.09775967), ('get', 0.09390698), ('audit', 0.07425884), ('ref', 0.073409006), ('hint', 0.06750496), ('appearance', 0.050742205), ('item', 0.042754043), ('reference', 0.035800155), ('log', 0.021714337)]), (18, [('note', 0.09356384), ('discrepa

In [17]:
import warnings
# we use an old library for generating this visualization
warnings.filterwarnings("ignore")

import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_new, id2word, mds="mmds", R=30)
vis

/home/markus/apps/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/markus/apps/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/markus/apps/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/markus/apps/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/markus/apps/anaconda3/lib/python3.9/site-packages/past/builtins/mi

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
2      0.333379  0.056039       1        1  5.218078
3     -0.239769  0.123863       2        1  4.846497
5     -0.425789 -0.085758       3        1  4.613144
47     0.062773  0.213497       4        1  4.570249
15    -0.025902  0.396625       5        1  3.642362
14    -0.048858  0.188244       6        1  3.507180
24    -0.075297  0.047467       7        1  3.496403
38    -0.130606 -0.037176       8        1  3.461880
34     0.467760 -0.127765       9        1  3.352823
18     0.304575  0.083905      10        1  3.252509
32     0.307882  0.206461      11        1  3.187664
44     0.406600 -0.084750      12        1  2.941435
26    -0.198936 -0.074080      13        1  2.593295
17     0.101157  0.113436      14        1  2.509922
49    -0.381309  0.132184      15        1  2.409307
42    -0.227254 -0.191631      16        1  2.393679
1     -0.331880 -0.193287      17        1  2.317957
20     0.209911  0.146708      18        1  2.282402
40     0.163722  0.282247      19        1  2.167517
31    -0.125342 -0.176482      20        1  2.104716
19     0.040950 -0.154774      21        1  1.982665
30     0.158306  0.031923      22        1  1.972867
35    -0.189165  0.217674      23        1  1.948609
16    -0.304580  0.074961      24        1  1.757286
39    -0.086400  0.304928      25        1  1.736804
23    -0.307269  0.282143      26        1  1.727034
25     0.077122 -0.000811      27        1  1.574804
11    -0.345577  0.198786      28        1  1.562986
13     0.186787 -0.087801      29        1  1.549301
48    -0.156651  0.054131      30        1  1.509521
43    -0.004609 -0.072633      31        1  1.496725
12     0.201007  0.409224      32        1  1.465130
37     0.076696  0.362719      33        1  1.435589
27    -0.318771 -0.268876      34        1  1.422229
6      0.181191 -0.259035      35        1  1.362835
0     -0.226394 -0.290566      36        1  1.325363
41     0.297942 -0.293921      37        1  1.012447
4      0.184274 -0.434385      38        1  0.937456
28     0.276616  0.276888      39        1  0.826413
45    -0.027113 -0.271505      40        1  0.782956
8     -0.143986  0.510292      41        1  0.769196
29     0.322547 -0.212443      42        1  0.742800
10     0.463836  0.119869      43        1  0.702428
33    -0.426133 -0.013631      44        1  0.677088
22     0.025111 -0.317906      45        1  0.615254
46     0.069128 -0.452630      46        1  0.599283
9     -0.173568  0.318979      47        1  0.528201
7      0.264848 -0.179735      48        1  0.527230
36    -0.091132 -0.448621      49        1  0.328608
21    -0.171827 -0.422991      50        1  0.251872, topic_info=        Term         Freq        Total Category  logprob  loglift
16      item  1252.000000  1252.000000  Default  30.0000  30.0000
10     event  1172.000000  1172.000000  Default  29.0000  29.0000
30     study  1768.000000  1768.000000  Default  28.0000  28.0000
370     rule  1333.000000  1333.000000  Default  27.0000  27.0000
18      name  1506.000000  1506.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
61   integer     1.049740   436.179167  Topic50  -4.9069  -0.0455
258   append     0.046060    31.231144  Topic50  -8.0332  -0.5352
101    query     0.091572   333.408745  Topic50  -7.3461  -2.2160
39      list     0.087561   765.214175  Topic50  -7.3908  -3.0916
3      class     0.063233   478.634963  Topic50  -7.7164  -2.9479

[2378 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
780       1  0.020042  abstract
780       4  0.020042  abstract
780       5  0.220460  abstract
780       8  0.010021  abstract
780      18  0.030063  abstract
...     ...       ...       ...
1003     41  0.120921      zero
596       6  0.517095       zip
1025      6  0.262991    zipped
1025     45  0.5259